In [1]:
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
import glob
import pandas as pd
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/8bit/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                                        words_level=True, 
                                                        similarity_types=['case', 'stop_words', 'subset', 'superset'])#,'leveshtein'])
                    # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [4]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 60].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
482,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,70,0.635337,0.614908,0.657171
480,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,80,0.635337,0.614908,0.657171
478,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,90,0.635337,0.614908,0.657171
485,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,100,0.635337,0.614908,0.657171
484,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,60,0.635337,0.614908,0.657171
...,...,...,...,...,...,...
598,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.008748,0.033635,0.005028
599,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,70,0.008748,0.033635,0.005028
600,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,65,0.008748,0.033635,0.005028
601,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,60,0.008748,0.033635,0.005028


In [15]:
## evaluation_results.to_csv('data/mistral/evaluation_results/results.csv', index=False)

In [23]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 100].sort_values(by='f1_score', ascending=False)['file'].to_list()[0:5]

['data/test_data_processed/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_8_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_4_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_4_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_4_0.0002.csv',
 'data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_8_0.0002.csv']

### One Run

In [5]:
file ='data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_8_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=False, similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein'])


{'evaluation':      TP  FP  FN
 0     5   1   5
 1     4   0   0
 2     3   0   1
 3     4   0   1
 4     2   3   0
 ..   ..  ..  ..
 676   4   0   0
 677   4   1   0
 678   3   2   0
 679   3   1   2
 680   3   3   1
 
 [681 rows x 3 columns],
 'precision': 0.7872415001752541,
 'recall': 0.6090021691973969,
 'f1': 0.6867451460021403}

In [4]:
file ='data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_4_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
                                    words_level=True, similarity_types=['case', 'stop_words', 'subset', 'superset'])


{'evaluation':      TP  FP  FN
 0     7   9   7
 1     4   8   0
 2     4   8   4
 3     5   8   1
 4     2  11   0
 ..   ..  ..  ..
 676   2  10   2
 677   3   8   2
 678   2   9   3
 679   6   8   4
 680   2   9   6
 
 [681 rows x 3 columns],
 'precision': 0.3156546793298474,
 'recall': 0.5075497597803706,
 'f1': 0.3892361339561814}

### BASE MODEL MISTRAL 8bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/8bit_base/*.csv') 
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [2]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 80].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
133,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,80,0.333916,0.260224,0.465835
132,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,85,0.321196,0.250189,0.448482
131,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,90,0.312688,0.243325,0.437364
130,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,95,0.309780,0.241062,0.433297
138,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,100,0.308811,0.240308,0.431941
...,...,...,...,...,...,...
19,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,False,100,0.128506,0.281651,0.083243
31,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,90,0.127428,0.085827,0.247289
30,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,95,0.127009,0.085545,0.246475
38,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,100,0.126729,0.085357,0.245933


### BASE MODEL MISTRAL 4bit

In [2]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

evaluators = {}
csv_files = glob.glob('data/mistral/4bit_base/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    if not file.strip().endswith('BaseModel.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference0_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference4_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference2_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference3_BaseModel.csv
FILE:  data/mistral/4bit_base/maxNewTokensFactor8_nShotsInference1_BaseModel.csv


In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 80].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
13,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.466546,0.445486,0.489696
23,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.463026,0.446798,0.480477
33,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.461848,0.456885,0.466920
22,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,85,0.451200,0.434859,0.468818
12,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,85,0.446995,0.426217,0.469902
32,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,85,0.445845,0.440880,0.450922
21,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,90,0.442473,0.426345,0.459870
43,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,80,0.441708,0.445856,0.437636
20,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,95,0.440010,0.423869,0.457430
28,data/mistral/4bit_base/maxNewTokensFactor8_nSh...,True,100,0.438910,0.422758,0.456345


In [4]:
##evaluation_results.to_csv('data/evaluation_results/mistral_4bit_base.csv', index=False)

### MISTRAL BASE 8bit

In [5]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

evaluators = {}
csv_files = glob.glob('data/mistral/8bit_base/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    if not file.strip().endswith('BaseModel.csv'):
        continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference0_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor2_nShotsInference0_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference4_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor2_nShotsInference4_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference1_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference2_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference2_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference4_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference3_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor4_nShotsInference0_BaseModel.csv
FILE:  data/mistral/8bit_base/maxNewTokensFactor8_nShotsInference3_BaseMo

In [6]:
evaluation_results[evaluation_results['similar_is_equal_threshold'] >= 80].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
133,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,80,0.333916,0.260224,0.465835
132,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,85,0.321196,0.250189,0.448482
131,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,90,0.312688,0.243325,0.437364
130,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,95,0.309780,0.241062,0.433297
138,data/mistral/8bit_base/maxNewTokensFactor8_nSh...,True,100,0.308811,0.240308,0.431941
...,...,...,...,...,...,...
19,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,False,100,0.128506,0.281651,0.083243
31,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,90,0.127428,0.085827,0.247289
30,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,95,0.127009,0.085545,0.246475
38,data/mistral/8bit_base/maxNewTokensFactor2_nSh...,True,100,0.126729,0.085357,0.245933


In [7]:
##evaluation_results.to_csv('data/evaluation_results/mistral_8bit_base.csv', index=False)

### MISTRAL 8bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/mistral/8bit/*.csv') #'data/mistral/test_data_processed/*.csv'
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    # if file.strip().endswith('0.002.csv'):
    #     continue
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break

In [8]:
##evaluation_results.to_csv('data/evaluation_results/mistral_8bit.csv', index=False)

In [5]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
535,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.648048,0.633049,0.663774
54,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.641816,0.602763,0.686280
555,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.638147,0.598386,0.683568
164,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,75,0.638144,0.631369,0.645065
534,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,80,0.631801,0.615860,0.648590
...,...,...,...,...,...,...
669,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,100,0.005189,0.019928,0.002983
663,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,85,0.005189,0.019928,0.002983
662,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,90,0.005189,0.019928,0.002983
661,data/mistral/8bit/maxNewTokensFactor8_nShotsIn...,True,95,0.005189,0.019928,0.002983


### LLAMA 13 4bit

In [7]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/13B_4bit/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=100,
                                                words_level=True,
                                                similarity_types=['case', 'stop_words', 'subset', 'superset'])
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
            evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}

        elif not similar_is_equal:
            evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100,
                                                words_level=True,
                                                similarity_types=['case', 'stop_words', 'subset', 'superset'])
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            # print('DONE')



Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_4_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference4_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_2_0.0008.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference4_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_8_0.0002.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_8_0.0008.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference2_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_4_0.0008.csv
FILE:  data/llama/13B_4bit/maxNewTokensFactor8_nShotsInference4_Llama-2-13b-chat-hf_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_2_0.0002.csv
FILE:  data/llama/1

In [8]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
82,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,True,100,0.481901,0.368749,0.695238
26,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,True,100,0.478199,0.368759,0.680013
40,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,True,100,0.472521,0.366797,0.663874
114,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,True,100,0.450015,0.330673,0.704145
42,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,True,100,0.448366,0.333982,0.681909
...,...,...,...,...,...,...
63,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,False,100,0.215044,0.142909,0.434225
5,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,False,100,0.207913,0.138867,0.413521
131,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,False,100,0.207215,0.138471,0.411506
105,data/llama/13B_4bit/maxNewTokensFactor8_nShots...,False,100,0.206795,0.137973,0.412605


In [ ]:
##evaluation_results.to_csv('data/evaluation_results/llama13B_4bit.csv', index=False)

### LLAMA 8bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/8bit/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [8]:
evaluator.data[1]

{'sentence': 'Hormonal study and dynamic biochemical tests performed indicated ECS.',
 'entities': "[{'id': '1704', 'offsets': array([ 9, 14]), 'role': '', 'semantic_type_id': '', 'text': 'study', 'type': 'EVENT'}\n {'id': '1719', 'offsets': array([39, 44]), 'role': '', 'semantic_type_id': '', 'text': 'tests', 'type': 'EVENT'}\n {'id': '1734', 'offsets': array([55, 64]), 'role': '', 'semantic_type_id': '', 'text': 'indicated', 'type': 'EVENT'}\n {'id': '1749', 'offsets': array([65, 68]), 'role': '', 'semantic_type_id': '', 'text': 'ECS', 'type': 'EVENT'}]",
 'original_text': 'A 46-year-old man with hypertension and dyslipidemia diagnosed 4-months before, as well as new-onset diabetes mellitus unveiled 1-month earlier, was referred to emergency department for hypokalemia. Hormonal study and dynamic biochemical tests performed indicated ECS. Imaging and cytological findings pointed toward a likely primary right parotid malignancy with liver metastases. Somatostatin receptor scintigraphy 

In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
354,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,75,0.405388,0.277098,0.754881
104,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,75,0.404637,0.276725,0.752440
353,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,80,0.398314,0.272060,0.743221
374,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,75,0.396249,0.271486,0.733189
103,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,80,0.396191,0.270660,0.738883
...,...,...,...,...,...,...
272,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,85,0.250214,0.182912,0.395879
271,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,90,0.248097,0.181227,0.393167
270,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,95,0.247070,0.180477,0.391540
278,data/llama/8bit/maxNewTokensFactor8_nShotsInfe...,True,100,0.246536,0.180080,0.390727


In [4]:
##evaluation_results.to_csv('data/evaluation_results/llama7b_8bit.csv', index=False)

### LLAM7 BASE 4bit

In [ ]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/7B_4bit_base/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


In [3]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
144,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,75,0.271827,0.230436,0.331345
143,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,80,0.261285,0.221447,0.318601
142,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,85,0.254501,0.215631,0.310466
141,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,90,0.248140,0.210160,0.302874
140,data/llama/7B_4bit_base/maxNewTokensFactor8_nS...,True,95,0.247668,0.209744,0.302332
...,...,...,...,...,...,...
12,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,85,0.003781,0.466667,0.001898
13,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,80,0.003781,0.466667,0.001898
18,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,100,0.003781,0.466667,0.001898
10,data/llama/7B_4bit_base/maxNewTokensFactor2_nS...,True,95,0.003781,0.466667,0.001898


In [4]:
##evaluation_results.to_csv('data/evaluation_results/llama7b_4bit_base.csv', index=False)

### LLAMA7 BASE 8bit

In [5]:
import glob
import pandas as pd
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list
#adapters_list = generate_ft_adapters_list("enlayer1_3epochs_4bits__ft_params")
evaluators = {}
csv_files = glob.glob('data/llama/7B_8bit_base/*.csv') 
#csv_files = ['data/test_data_processed/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv']
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

print(evaluation_results)
for file in csv_files:
    print("FILE: " , file)
    eval_data = Dataset.from_csv(file) 
    output_cleaner = OutputCleaner()
    cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
    for similar_is_equal in similar_is_equal_list:
        if similar_is_equal:
            for similar_is_equal_threshold in similar_is_equal_threshold_list:
                # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                                
                try:
                    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                    evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal_threshold, similar_is_equal_threshold=similar_is_equal_threshold)
                    #evaluators[f"{file}_SimilarIsEqual{similar_is_equal}_Threshold{similar_is_equal_threshold}"] = evaluator
                    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                    # print('DONE')
                except:
                    break
        elif not similar_is_equal:
            # evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
            # evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
            # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
            # evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            
            try:
                evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
                evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=100)
                # evaluators[f"{file}_SimilarIsEqual{similar_is_equal}"] = evaluator
                evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': similar_is_equal, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                # print('DONE')
            except:
                #print('SKIPPING THIS')
                break


Empty DataFrame
Columns: [file, similar_is_equal, similar_is_equal_threshold, f1_score, precision, recall]
Index: []
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference0_BaseModel.csv
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference2_BaseModel.csv
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference3_BaseModel.csv
FILE:  data/llama/7B_8bit_base/maxNewTokensFactor8_nShotsInference1_BaseModel.csv


In [6]:
evaluation_results[evaluation_results['similar_is_equal_threshold']>=75].sort_values(by='f1_score', ascending=False)

,file,similar_is_equal,similar_is_equal_threshold,f1_score,precision,recall
34,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,75,0.287719,0.244096,0.350325
4,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,75,0.285771,0.532939,0.195228
33,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,80,0.276906,0.234850,0.337310
3,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,80,0.271771,0.506282,0.185738
32,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,85,0.271473,0.230189,0.330803
31,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,90,0.265436,0.225052,0.323482
30,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,95,0.264457,0.224170,0.322397
38,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,100,0.263790,0.223605,0.321584
2,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,85,0.257732,0.479351,0.176247
1,data/llama/7B_8bit_base/maxNewTokensFactor8_nS...,True,90,0.252131,0.468681,0.172451


In [7]:
##evaluation_results.to_csv('data/evaluation_results/llama7b_8bit_base.csv', index=False)

### MISTRAL BASE simplest_prompt one run

In [4]:
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

similar_is_equal_list = postprocessing.similar_is_equal_list
similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list

file = 'data/test_data_processed/maxNewTokensFactor6_nShotsInference0_Mistral-7B-v0.1_simplest_prompt_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_4_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 80
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator_mistral_base = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator_mistral_base.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold)


file = 'data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_4_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 80
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

evaluator_mistral_instruct = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
evaluator_mistral_instruct.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold)

print(evaluator_mistral_instruct.evaluation_table)
print(evaluator_mistral_base.evaluation_table)



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1050.15it/s]
Generating train split: 681 examples [00:00, 20161.65 examples/s]
Map: 100%|██████████| 681/681 [00:00<00:00, 6854.36 examples/s]


{'evaluation':      TP  FP  FN
0     0   0  10
1     0   1   4
2     0   0   4
3     5   0   0
4     1   0   1
..   ..  ..  ..
676   0   0   4
677   3   0   1
678   3   0   0
679   3   1   2
680   0   0   4

[681 rows x 3 columns], 'precision': 0.7130730050933786, 'recall': 0.34164859002169196, 'f1': 0.46196150320806595}
{'evaluation':      TP  FP  FN
0     2   0   8
1     3   0   1
2     2   1   2
3     3   0   2
4     2   1   0
..   ..  ..  ..
676   2   1   2
677   3   0   1
678   2   1   1
679   2   1   3
680   3   0   1

[681 rows x 3 columns], 'precision': 0.6405353728489483, 'recall': 0.2725054229934924, 'f1': 0.3823473463952825}


### MISTRAL 4bit

In [2]:
import json
import re
from typing import Tuple
from typing import List

class OutputCleaner():
    def __init__(self) -> None:
        pass
  
    def _remove_space_from_dict_keys(self, model_ouput_list: list) -> list:
        """
        Remove the spaces from the keys of a dictionary. E.g., [{"entity ": "value"}] -> [{"entity": "value"}]

        Args:
        model_ouput_list (dict): the list of dictionaries to be cleaned

        return:
        list: the cleaned list of dicts
        """
        out = []
        for dict in model_ouput_list:
            out.append({k.replace(' ', ''): v for k, v in dict.items()})
        return out
    
    def _drop_duplicates(self, model_response: list) -> str:
        """
        Drop the duplicates from a list. This is useful when the model output contains the same entity multiple times.

        Args:
        model_response (str): the model response with no duplicates
        """
        try :
            return list({v['entity']:v for v in model_response}.values())
        except Exception as error:
            model_response = self._remove_space_from_dict_keys(model_response)
            return list({v['entity']:v for v in model_response}.values())
        
    def _assess_model_output(self, model_response: str) -> bool:
        """
        Check if the model output is in the right format. If not, return False.
        
        Args:
        model_output (str): the postprocessed model output after beeing passed to _postprocess_model_output()

        return:
        bool: True if the format is correct, False otherwise
        """
        good_format = True
        try :
            res = json.loads(model_response)
            # print( res)
        except:
            good_format = False
        return good_format

            
    def _remove_json_special_chars(self, string):
        chars = ['\xa0']
        for char in chars:
            string = string.replace(char, ' ')
        return string
    
    def _clean_ground_truth(self, example: dict) -> dict:
        ground_truth = example['ground_truth']
        ground_truth = self._remove_json_special_chars(ground_truth)
        if ground_truth == ']':
            ground_truth = '[' + ground_truth
        return({'ground_truth': ground_truth})

    def _clean_model_output(self, example: dict,  wrong_keys_to_entity:bool, latest_version:bool=True) -> dict:
        """
        Postprocess the model output to return a json like formatted string that can be used to compute the F1 score.

        Args:
        model_output (str): the model output as it is returned by the model. The processing of the output is done in the function
        wrong_keys_to_entity (bool): if True, the function also extracts the dictionaries with keys different from 'entity', converting the keys into 'entity'. If not, all keys that are not 'entity' are dropped

        return:
        dict: the model response

        """
        def has_unclosed_square_brackets(s:str)  -> bool:
            count = 0
            for char in s:
                if char == '[':
                    count += 1
                elif char == ']':
                    count -= 1
                    if count < 0:
                        return True
            return count > 0
        
        def has_unopen_square_brackets(s:str)  -> bool:
            count = 0
            for char in s:
                if char == '[':
                    count -= 1
                elif char == ']':
                    count += 1
                    if count > 0:
                        return True
            return count > 0
        
        def is_empty_list(string:str)  -> bool:
            if string=='[]':
                return True
            return False
        
        def is_list_of_lists(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, list) for item in tmp):
                    return True
            return False
        
        def is_list_of_dicts(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, dict) for item in tmp):
                    return True
            return False
        
        def is_list_of_lists_and_dict(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                found_dict = False
                found_list = False
                for element in tmp:
                    if isinstance(element, list):
                        found_list = True
                    elif isinstance(element, dict):
                        found_dict = True
                    if found_list and found_dict:
                        return True
            return False
        
        def is_list_of_strings(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, list) and all(isinstance(item, str) for item in tmp):
                    return True
            return False

        def is_list_of_dicts_and_strings(string:str)  -> bool:
            if self._assess_model_output(string):
                #print('ASSESSED')
                tmp = json.loads(string)
                found_dict = False
                found_string = False
                for element in tmp:
                    if isinstance(element, str):
                        found_string = True
                    elif isinstance(element, dict):
                        found_dict = True
                    if found_string and found_dict:
                        return True
            return False
        
        def is_string(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, str):
                    return True
            return False
        
        def is_numeric(string:str)  -> bool:
            if self._assess_model_output(string):
                tmp = json.loads(string)
                if isinstance(tmp, (int, float)):
                    return True
            return False
        
        def are_entities_extracted_as_dict_keys_instead_of_values(string:str, example:dict) -> bool:
            if is_list_of_dicts(string):
                tmp = json.loads(string)
                keys = [key for item in tmp for key in item.keys()]
                if 'entity' not in keys:
                    if all(entity in example['sentence'] for entity in keys):
                        return True
            return False
        
        
        
        def convert_wrong_keys_into_entity(string:str) -> List[str]:
            if is_list_of_dicts(string):
                tmp = json.loads(string)
                tmp = [str({"entity":v}) for el in tmp for v in el.values()]
                return tmp
            else:
                return []


        def only_dicts_with_key_entity(string:str, wrong_keys_to_entity:bool) -> Tuple[bool, str]:
            """
            Extract only the dictionaries with the key 'entity' in the list of dictionaries in the string
            
            Args:
            string (str): the string to be cleaned
            wrong_keys_to_entity (bool): if True, the function also extracts the dictionaries with keys different from 'entity', converting the keys into 'entity'
            """
            els_between_curly = re.findall(r'\{(.+?)\}', string)
            clean = [el for el in els_between_curly if el.startswith('"entity"') or el.startswith("'entity'")]
            clean = ['{' + el + '}' for el in clean]
            dirty = []
            if wrong_keys_to_entity:
                dirty = [el for el in els_between_curly if (not el.startswith('"entity"')) and (not el.startswith("'entity'"))]
                dirty = ['{' + el + '}' for el in dirty]
                dirty = '[' + ', '.join(dirty) + ']'
                cleaned_dirty = convert_wrong_keys_into_entity(dirty)
                out = '[' + ', '.join(clean) + ', '.join(cleaned_dirty) +  ']'
            else:
                out = '[' + ', '.join(clean) + ']'
            out = out.replace("{\'", "{\"").replace("\'}", "\"}").replace("\'ent", "\"ent").replace("ty\'", "ty\"").replace(" \'", " \"")
            operations_performed = False
            if len(clean) != len(els_between_curly):
                operations_performed = True
            if is_empty_list(out):
                return operations_performed, '[{"entity":""}]'
            return operations_performed, str(out)
        

        model_output = example['model_responses']        
        print('TESTTT')
        print('ORIGINAL MODEL OUTPUT: ', model_output)
        
        if model_output is None or is_empty_list(model_output):
            return {'model_output':'[{"entity":""}]'}
        
        model_output = self._remove_json_special_chars(model_output)
                
        if are_entities_extracted_as_dict_keys_instead_of_values(model_output, example):
            print('ENTITIES EXTRACTED AS DICT KEYS INSTEAD OF VALUES')
            tmp = json.loads(model_output)
            tmp = [{"entity":k} for el in tmp for k in el.keys() ]
            tmp = str(tmp)
            return {'model_output':tmp}

        if is_numeric(model_output):
            print('IS NUMERIC')
            return {'model_output':'[{"entity":""}]'}
        
        if is_list_of_lists_and_dict(model_output):
            print('is_list_of_lists_and_dict')
            tmp = json.loads(model_output)
            for el in tmp:
                if isinstance(el, list):
                    tmp = str(el)
                    # print('is_list_of_lists_and_dict')
                    return {'model_output':tmp}
                
        if is_list_of_lists(model_output):
            print('is_list_of_lists')
            tmp = json.loads(model_output)
            tmp2 = str(tmp[0]).replace("'", "\"")
            if is_list_of_dicts_and_strings(tmp2):
                tmp = tmp[0]
                out = [item for item in tmp if isinstance(item, dict)]
                print('questo:', out)
                
                return {'model_output':str(out)} 
            tmp = str(tmp[0])
            return {'model_output':tmp}

        if is_list_of_strings(model_output):
            print('is_list_of_strings')
            tmp = json.loads(model_output)
            tmp = [{"entity":el} for el in tmp]
            tmp = str(tmp)
            # print('is_list_of_strings')
            return {'model_output': tmp}
        
        if is_string(model_output):
            # model_output = model_output.replace("{\'", "{\"").replace("\'}", "\"}").replace("\'ent", "\"ent").replace("ty\'", "ty\"").replace(" \'", " \"")
            print('PULITO: ', model_output)
            tmp = json.loads(model_output)
            if all(el in tmp for el in ['{', 'entity', '}']):
                return {'model_output':tmp}
            tmp = [{"entity":tmp}]
            tmp = str(tmp)
            #print('is_string')
            return {'model_output':tmp}

        
        if latest_version:
            model_output = self._extract_text_between_curl_brackets(model_output)
            #last attempt to clean 
            model_output = self._clean_text_between_curl_brackets(model_output)
            cleaning_done, cleaned_model_output = only_dicts_with_key_entity(model_output, wrong_keys_to_entity=wrong_keys_to_entity)
            if cleaning_done:
                model_output = cleaned_model_output
            
            if is_list_of_dicts(model_output):
                tmp = json.loads(model_output)
                #tmp = [item for item in tmp if 'entity' in item.keys()]
                return {'model_output':str(tmp)}
            
            else: 
                # print('NOT CLEANED: ', model_output, '\n\n')
                return {'model_output':'[{"entity":""}]'}
        

        if not latest_version:
            if has_unopen_square_brackets(model_output):
                last_bracket_index = model_output.rfind('],') # keep the closed list
                model_output = '[' + model_output[:last_bracket_index+1] 
                return {'model_output':model_output} 
            
            tmp = re.findall(r'\[\{(.+?)\}\]', model_output)
            if len(tmp) != 0:
                tmp = '[{' + tmp[0] + '}]'
                if self._assess_model_output(tmp):
                    return {'model_output':tmp}
                
            if has_unclosed_square_brackets(model_output):
                last_bracket_index = model_output.rfind('},') # find the last complete entity
                model_output = model_output[:last_bracket_index+1] + ']' 
                return {'model_output':model_output} 


            if model_output.strip()[0] == '{':
                tmp = '[' + model_output + ']'
                if self._assess_model_output(tmp):
                    return {'model_output':tmp}
                else:
                    last_bracket_index = model_output.rfind('},') # find the last complete entity
                    model_output = '[' + model_output[:last_bracket_index+1] + ']'
                    return {'model_output':model_output}
                
            if model_output.strip().startswith('[['):
                tmp = model_output[1:]
                if self._assess_model_output(tmp):
                    return {'model_output':tmp}

            # print('THIS IS A BROKEN ROW: ', model_output)

            return {'model_output':model_output}
    
    def _extract_text_between_curl_brackets(self, model_output: str) -> str:
        """
        Extract the text between the curl brackets of the model output, as enitties are usually outputted in this format: {"entity": "value"}

        Args:
        model_output (str): the example from the dataset

        """
        text_between_curl_brackets = re.findall(r'\{(.+?)\}', model_output)
        cleaned_output = ['{'+ el +'}' for el in text_between_curl_brackets]
        cleaned_output = '[' + ', '.join(cleaned_output) + ']'
        return cleaned_output
    

    def _clean_text_between_curl_brackets(self, text_between_curl_brackets: str) -> str:
        """
        Clean the text between the curl brackets of the model output, as entities are usually outputted in this format: {"key": "value"}

        Args:
        model_output (str): the example from the dataset

        """
        text_between_curl_brackets = re.sub(r'",(.+?)}', r'"}', text_between_curl_brackets)
        return(text_between_curl_brackets)

        # list_of_dicts = json.loads(text_between_curl_brackets)
        # list_of_dicts_str = [str(item) for item in list_of_dicts]
        # out = []
        # print('list_of_dicts: ', list_of_dicts_str)
        # if ' '.join(list_of_dicts_str).count(':') > len(list_of_dicts):
        #     for item in list_of_dicts:
        #         if len(item) > 1:
        #             item = self._keep_only_one_keyVal_from_dict(item)
        #         out.append(item)
        #return str(out)

    # def _keep_only_one_keyVal_from_dict(dict: dict) -> dict:
    #     """
    #     Keep only one key-value pair from a dictionary. This is useful when the model outputs a dictionary with more than one key-value pair.

    #     Args:
    #     dict (dict): the dictionary to be cleaned

    #     return:
    #     dict: the cleaned dictionary
    #     """
    #     k_first = "entity"
    #     if k_first not in dict.keys():
    #         k_first = list(dict.keys())[0]
    #     return {k_first:dict[k_first]}

    
    def apply_cleaning(self, data, wrong_keys_to_entity) -> None:
        """
        Apply the cleaning to the model output and return the cleaned response in a new cloumn called 'model_output

        Args:
        model_output (str): the model output as it is returned by the model. The processing of the output is done in the function
        wrong_keys_to_entity (bool): if True, the function also extracts the dictionaries with keys different from 'entity', converting the keys into 'entity'. If not, all keys that are not 'entity' are dropped

        return:
        str: the model response, i.e. the model output without the instruction
        """
        data = data.map(lambda x: self._clean_ground_truth(x), remove_columns=['ground_truth'])
        data = data.map(lambda x: self._clean_model_output(x, wrong_keys_to_entity)) 
        return data

In [1]:
from datasets import Dataset
from config import postprocessing
from utils.evaluator import Evaluator
from utils.output_cleaner import OutputCleaner

file ='data/mistral/4bit/maxNewTokensFactor8_nShotsInference4_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv'
eval_data = Dataset.from_csv(file) 
#display(eval_data.to_pandas().head(3))
output_cleaner = OutputCleaner()
similar_is_equal = True
similar_is_equal_threshold = 100
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)#.select(range(138,139))

# evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
# evaluator.generate_evaluation_table(similar_is_equal=similar_is_equal, similar_is_equal_threshold=similar_is_equal_threshold,
#                                     words_level=False, similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein'])


/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pferrazzi/miniconda3/envs/lm_finetune_env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [17]:
import glob
import pandas as pd

file ='data/mistral/4bit/maxNewTokensFactor8_nShotsInference4_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv'
evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])
input_data_path = postprocessing.input_data_dir_path
output_data_path = postprocessing.output_data_path
words_level = postprocessing.words_level
similarity_types=['case', 'stop_words', 'subset', 'superset', 'leveshtein']


print("FILE: " , file)
eval_data = Dataset.from_csv(file) 
output_cleaner = OutputCleaner()
cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=False)
for similar_is_equal_threshold in [85, 100]:
    evaluator = Evaluator(data=cleaned_data, offset=False, output_cleaner=output_cleaner)
    evaluator.generate_evaluation_table(similar_is_equal=True, 
                                        similar_is_equal_threshold=similar_is_equal_threshold,
                                        words_level=words_level, 
                                        similarity_types=similarity_types)
    evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': True, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
            


FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference4_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv


In [ ]:
def evaluate(input_data_path:str, 
             output_file_path:str, 
             words_level:bool,
             similar_is_equal_threshold_list,
             similarity_types:list = ['case', 'stop_words', 'subset', 'superset'],
             wrong_keys_to_entity=False,
             offset=False) -> pd.DataFrame:
    """
    Evaluate the model on the test data
    
    Args:
    input_data_path: The path to the test data directory
    output_data_path: The path to the output file
    
    Returns:
    evaluation_results: The evaluation results
    """
    csv_files = glob.glob(input_data_path + '/*.csv') 
    evaluation_results = pd.DataFrame(columns=['file', 'similar_is_equal', 'similar_is_equal_threshold', 'f1_score', 'precision', 'recall'])

    for file in csv_files:
        print("FILE: " , file)
        eval_data = Dataset.from_csv(file) 
        output_cleaner = OutputCleaner()
        cleaned_data = output_cleaner.apply_cleaning(eval_data, wrong_keys_to_entity=wrong_keys_to_entity)
        for similar_is_equal_threshold in similar_is_equal_threshold_list:
            evaluator = Evaluator(data=cleaned_data, offset=offset, output_cleaner=output_cleaner)
            evaluator.generate_evaluation_table(similar_is_equal=True, 
                                                similar_is_equal_threshold=similar_is_equal_threshold,
                                                words_level=words_level, 
                                                similarity_types=similarity_types)
            evaluation_results.loc[len(evaluation_results)] = {'file': file, 'similar_is_equal': True, 'similar_is_equal_threshold': similar_is_equal_threshold, 'f1_score': evaluator.evaluation_table['f1'], 'precision': evaluator.evaluation_table['precision'], 'recall': evaluator.evaluation_table['recall']}
                
    evaluation_results.to_csv(output_file_path, index=False)
    return evaluation_results

if True:
    input_data_path = 'data/mistral/4bit'
    output_data_path = 'data/example.csv'
    words_level = postprocessing.words_level
    evaluate(input_data_path, output_data_path, words_level, 
             similar_is_equal_threshold_list = postprocessing.similar_is_equal_threshold_list,
             similarity_types = ['case', 'stop_words', 'subset', 'superset'],
             wrong_keys_to_entity = False,
             offset = False)


In [2]:
from evaluation import evaluate


res = evaluate('data/mistral/4bit',
         'data/evaluation_results/mistral_4bit.csv',
         words_level=True,
         similar_is_equal_threshold_list=[80, 100],
         similarity_types=['case', 'stop_words', 'subset', 'superset'],
         wrong_keys_to_entity=False,
         offset=False)


FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_8_0.002.csv


/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])
/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])


FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_4_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.01_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.01_4_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_4_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.01_8_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_4_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bflo

/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])
/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])


FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_4_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_8_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.01_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference4_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_8_0.002.csv


...

/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])
/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])


FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.01_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.05_8_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_4_0.002.csv


/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])
/home/pferrazzi/mistral_finetuning/utils/evaluator.py:445: RuntimeWarning: invalid value encountered in scalar divide
  precision = summary['TP'] / (summary['TP'] + summary['FP'])


FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_8_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference4_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_16_32_0.01_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_8_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.05_4_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_64_32_0.05_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor8_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bfloat16_32_32_0.01_2_0.0002.csv
FILE:  data/mistral/4bit/maxNewTokensFactor4_nShotsInference2_Mistral-7B-Instruct-v0.2_adapters_en.layer1_4_torch.bflo

JSONDecodeError: Expecting value: line 1 column 114 (char 113)